In [1]:
import torch
from OScINN import OScINN1D

# === Initialize the Conditional INN (cINN) ===
oscinn = OScINN1D(9, 100, 8, cuda=False)  
# Create a Conditional INN with:
# - Input dimension: 9 (defines the dimensionality of the input dataset; each sample has 9 features).
# - Conditional input dimension: 100 (dimensionality of the target property data; each condition has 100 features).
# - Number of blocks: 8 (number of invertible coupling blocks in the INN).
# - cuda: False (runs on CPU; set True to run on GPU).

device = 'cuda' if torch.cuda.is_available() else 'cpu'  
# Check if GPU is available; use 'cuda' if available, otherwise use 'cpu'.

# === Generate Random Data for Testing ===
c = c2 = torch.randn(50, 1, 100).to(device)  
# Generate random conditional data to simulate the target property.
# Shape: (50, 1, 100)
# - 50 samples, each with a single 100-dimensional condition.

x = torch.randn(50, 9).to(device)  
# Generate random input data to simulate structures to be mapped.
# Shape: (50, 9)
# - 50 samples, each with 9 features (same as the input dimension of the cINN).

# === Process Target Property Through the Conditional Network ===
c = oscinn.cond_net(c)  
# Pass the target property (c) through the `cond_net` to transform it into a learned representation.
# Shape after this step: (50, 100)

print('condition shape: ', c.shape)  
# Print the shape of the transformed condition for verification.

c = [c.squeeze() for i in range(8)]  
# Prepare the transformed condition for use in all 8 invertible blocks.
# The condition is repeated and squeezed to match the dimensionality expected by each block.

# === Forward Mapping: Input Data to Latent Space ===
z, jac_z = oscinn.cinn(x, c=c, rev=True)  
# Map the input data (x) to the latent space (z) conditioned on the transformed condition (c).
# - `rev=True` ensures the forward mapping direction (input → latent space).
# Outputs:
# - `z`: Latent space representation of the input data (Shape: 50 x 9).
# - `jac_z`: Log Jacobian determinant for the transformation.

print('latent space shape: ', z.shape)  
# Print the shape of the latent space for verification.

condition shape:  torch.Size([50, 1, 15])
latent space shape:  torch.Size([50, 9])


In [2]:
# === Test `eval_forward` Method ===

print('\nTest eval_forward...')
z, jac = oscinn.eval_forward([x, c2[:, 0]])  
# Evaluate the cINN in forward mode using the `eval_forward` method.
# Inputs:
# - x: Input data (Shape: 50 x 9).
# - c2[:, 0]: Raw target property data without additional dimension (Shape: 50 x 100).
# Outputs:
# - z: Latent space representation of x.
# - jac: Log Jacobian determinant.

print('latent space shape: ', z.shape)  
# Print the latent space shape after using eval_forward.

z_1D, jac_1D = oscinn.eval_forward([x[0], c2[0, 0]])  
# Evaluate the forward pass for a single input sample (1D case).
# Inputs:
# - x[0]: First input sample (Shape: 9).
# - c2[0, 0]: First target property (Shape: 100).
# Outputs:
# - z_1D: Latent space for the single input sample.
# - jac_1D: Log Jacobian determinant for the transformation.

print('latent 1d space shape: ', z_1D.shape)  
# Print the shape of the latent space for a single input sample.


Test eval_forward...
latent space shape:  torch.Size([50, 9])
latent 1d space shape:  torch.Size([9])


In [3]:
# === Test `eval_inverse` Method ===

print('\nTest eval_inverse...')
x_hat, jac = oscinn.eval_inverse([z, c2[:, 0]])  
# Perform the inverse mapping (latent space → input space) to reconstruct the input data.
# Inputs:
# - z: Latent space representation (Shape: 50 x 9).
# - c2[:, 0]: Raw target property (Shape: 50 x 100).
# Outputs:
# - x_hat: Reconstructed input data (Shape: 50 x 9, matches input data shape).
# - jac: Log Jacobian determinant.

print('x_hat shape: ', x_hat.shape)  
# Print the shape of the reconstructed input data.


Test eval_inverse...
Initial z shape: torch.Size([50, 9])
Initial c shape: torch.Size([50, 100])
Adjusted c shape before cond_net: torch.Size([50, 1, 100])
Processed c shape after cond_net: torch.Size([50, 1, 15])
Prepared c shape for each block: 8 torch.Size([50, 15])
Adjusted z shape: torch.Size([50, 9])
Generated x_hat shape: torch.Size([50, 9])
x_hat shape:  torch.Size([50, 9])


In [4]:
x_1D, jac_1D = oscinn.eval_inverse([z[0], c2[0, 0]])  
# Perform the inverse mapping for a single sample (1D case).
# Inputs:
# - z[0]: First latent space sample (Shape: 9).
# - c2[0, 0]: First target property (Shape: 100).
# Outputs:
# - x_1D: Reconstructed input data for a single sample.
# - jac_1D: Log Jacobian determinant.

print('x_hat 1d shape: ', x_1D.shape)  
# Print the shape of the reconstructed data for a single sample.

Initial z shape: torch.Size([9])
Initial c shape: torch.Size([100])
Adjusted c shape before cond_net: torch.Size([2, 1, 100])
Processed c shape after cond_net: torch.Size([2, 1, 15])
Prepared c shape for each block: 8 torch.Size([2, 15])
Adjusted z shape: torch.Size([2, 9])
Generated x_hat shape: torch.Size([2, 9])
x_hat 1d shape:  torch.Size([9])


In [5]:
# === Training the cINN ===

print('\nTest training...')
dataset = torch.utils.data.TensorDataset(x, c2[:, 0])  
# Create a PyTorch dataset combining input data (x) and target property (c2[:, 0]).

dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True, drop_last=False)  
# Create a DataLoader for batching the dataset.
# Batch size: 5 samples per batch.

oscinn.optimizer = torch.optim.Adam  
# Set the optimizer for training (Adam optimizer).

oscinn.optimizer_kwargs = {'lr': 0.001}  
# Set the learning rate for the optimizer.

print(oscinn.optimizer)  
# Print the optimizer settings for verification.

oscinn.train(dataloader, 2)  
# Train the cINN for 2 epochs using the provided DataLoader.


Test training...
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


In [12]:
print('\nGenerating multiple samples for a target property...')

# Define a target property
target_property = torch.randn(1, 100).to(device)  # Shape: [1, 100]
print("Initial target_property shape:", target_property.shape)

# Expand target property to match expected shape [Batch, Channels, Width]
target_property = target_property.unsqueeze(1)  # Shape: [1, 1, 100]
print("Expanded target_property shape for cond_net:", target_property.shape)

# Set cond_net to evaluation mode
oscinn.cond_net.eval()

# Process the target property through cond_net
processed_target = oscinn.cond_net(target_property)  # Shape: [1, cond_dim, Width]
print("Processed target_property shape (after cond_net):", processed_target.shape)

# Squeeze unnecessary dimensions while retaining batch compatibility
processed_target = processed_target.squeeze(-1)  # Shape: [1, cond_dim]
print("Processed target_property shape (after squeeze):", processed_target.shape)

# Prepare the condition for each block
processed_condition = [processed_target for _ in range(oscinn.num_of_blocks)]  # Repeat for all blocks
print("Processed_condition shapes for blocks:", [pc.shape for pc in processed_condition])

# Number of samples to generate
num_samples = 5
generated_samples = []

for _ in range(num_samples):
    # Sample a random latent vector z (Shape: [1, input_dim])
    z = torch.randn(1, oscinn.input_dim).to(device)
    print("Latent vector z shape:", z.shape)

    try:
        # Use eval_inverse to generate a new sample
        generated_sample, _ = oscinn.eval_inverse([z, processed_target])  # Pass z and processed condition
        generated_samples.append(generated_sample)
    except RuntimeError as e:
        print("\nError encountered while generating samples:")
        print(e)
        break

# Display the generated samples (if any)
if generated_samples:
    for i, sample in enumerate(generated_samples):
        print(f"Generated sample {i + 1}: {sample.cpu().numpy()}")



Generating multiple samples for a target property...
Initial target_property shape: torch.Size([1, 100])
Expanded target_property shape for cond_net: torch.Size([1, 1, 100])
Processed target_property shape (after cond_net): torch.Size([1, 1, 15])
Processed target_property shape (after squeeze): torch.Size([1, 1, 15])
Processed_condition shapes for blocks: [torch.Size([1, 1, 15]), torch.Size([1, 1, 15]), torch.Size([1, 1, 15]), torch.Size([1, 1, 15]), torch.Size([1, 1, 15]), torch.Size([1, 1, 15]), torch.Size([1, 1, 15]), torch.Size([1, 1, 15])]
Latent vector z shape: torch.Size([1, 9])
Initial z shape: torch.Size([1, 9])
Initial c shape: torch.Size([1, 1, 15])
Adjusted c shape before cond_net: torch.Size([1, 1, 1, 15])

Error encountered while generating samples:
Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 1, 15]


In [13]:
generated_samples

[]